In [ ]:
#Working with JSON fields

In [ ]:
#import libs (or think of this as downloading python 'Apps')
import psycopg2
from jettings import Jettings
import os

In [ ]:
#Get database credentials
password_file="~/database.secrets"

password_file=os.path.expanduser(password_file)
password_file=os.path.abspath(password_file)

if(not os.path.exists(password_file)):
    print("File doesn't exist!")
    my_settings=Jettings(password_file)
    my_settings.sets(['tutorial','db_host'],"*****")
    my_settings.sets(['tutorial','db_username'],"*****")
    my_settings.sets(['tutorial','db_password'],"*****")
    my_settings.sets(['tutorial','db_name'],"*****")
    my_settings.sets(['tutorial','db_schema'],"*****")
    my_settings.sets(['tutorial','editlock'],"*****")
else:
    my_settings=Jettings(password_file)
    print("File exists!")
    edit_lock=my_settings.exists(['tutorial','editlock'])
    if(edit_lock):
        message="Please add connection details to file: "+str(password_file)+"\nand delete the edit lock."
        print(message)
    else:
        db_host=my_settings.gets(['tutorial','db_host'])
        db_username=my_settings.gets(['tutorial','db_username'])
        db_password=my_settings.gets(['tutorial','db_password'])
        db_name=my_settings.gets(['tutorial','db_name'])
        db_schema=my_settings.gets(['tutorial','db_schema'])
        print("Connection credentials retrieved!")

In [ ]:
#Get connection to your database (or think of this as loging in to your DB)
conn = None
try:
    conn=psycopg2.connect(host=db_host,
                          user=db_username,
                          password=db_password,
                          database=db_name)
    print("Connecting to the database...")
    conn.autocommit = True
except (Exception, psycopg2.DatabaseError) as banana:
    print(banana)

In [ ]:
#Target schema and new table
existing_schema="zokitools"
new_table="orders"

In [ ]:
#Create a new table, with a JSON field.
#Create sql statement with f-strings.
create_table=f'''
CREATE TABLE IF NOT EXISTS {existing_schema}.{new_table}(
id SERIAL NOT NULL PRIMARY KEY,
info json NOT NULL
)
'''
print(create_table)

In [ ]:
#Ask the connection for a cursor, then ask the cursor to run your sql statement.
cur=conn.cursor()
cur.execute(create_table)

In [ ]:
######  INSERTING JSON DATA ########
####################################

In [ ]:
#Create a query with variables.
field_tuple_string='(info)'
json_string='{"customer":"John Doe","items":{"product":"Beer","qty":6}}'
#Create insert query using f-strings and the variables above...
insert_sql=f'''
INSERT INTO {existing_schema}.{new_table} {field_tuple_string} VALUES ('{json_string}')
'''
print(insert_sql)

In [ ]:
cur.execute(insert_sql)

In [ ]:
#Construct strings with escaped { }
insert_sql2=f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES ('{{"customer":"John Doe","items":{{"product":"Beer","qty":6}}}}')
'''
print(insert_sql2)

In [ ]:
cur.execute(insert_sql2)

In [ ]:
insert_sql3=f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES 
      ('{{ "customer": "Lily Bush", "items": {{"product": "Diaper","qty": 24}}}}'),
      ('{{ "customer": "Josh William", "items": {{"product": "Toy Car","qty": 1}}}}'),
      ('{{ "customer": "Mary Clark", "items": {{"product": "Toy Train","qty": 2}}}}');
'''
print(insert_sql3)

In [ ]:
cur.execute(insert_sql3)

In [ ]:
insert_sql4=f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES 
      ('{{ "customer": "Lily Bush", "country":"US" , "items": {{"product": "Diaper","qty": 24}}}}'),
      ('{{ "customer": "Josh William","country": "NL", "items": {{"product": "Toy Car","qty": 1}}}}'),
      ('{{ "customer": "Mary Clark","country": "UK", "items": {{"product": "Toy Train","qty": 2}}}}');
'''
print(insert_sql4)

In [ ]:
cur.execute(insert_sql4)

In [ ]:
#CREATE JSON READ QUERY
select_json=f'select * from {existing_schema}.{new_table} where info->>\'country\'=\'US\';'
print(select_json)

In [ ]:
cur.execute(select_json)
customers=cur.fetchall()

In [ ]:
for a_customer in customers:
    print(a_customer)

In [ ]:
#CREATE NESTED JSON QUERY
select_json2=f'select * from {existing_schema}.{new_table} where info->\'items\'->>\'product\'=\'Diaper\';'
print(select_json2)

In [ ]:
cur.execute(select_json)
customers=cur.fetchall()

In [ ]:
for a_customer in customers:
    print(a_customer)

In [ ]:
#IMPORTANT NOTES
#Using '->>' returns query result as text, which is usefull for comparison.
#Using '->' returns a json object, which helps you query deeper json values ( as shown above )

In [ ]:
### Working with booleans
insert_sql5=f'''
INSERT INTO {existing_schema}.{new_table} (info) VALUES 
      ('{{ "customer": "James Kirk", "starshipregistry":"NCC-1701" ,"customerdiscount":"true", "items": {{"product": "Diaper","qty": 24}}}}'),
      ('{{ "customer": "Pavel Chekov","starshipregistry": "NCC-1701","customerdiscount":"true", "items": {{"product": "Toy Car","qty": 1}}}}'),
      ('{{ "customer": "Hikaru Sulu","starshipregistry": "NCC-1701","customerdiscount":"true", "items": {{"product": "Toy Train","qty": 2}}}}');
'''
print(insert_sql5)

In [ ]:
cur.execute(insert_sql5)

In [ ]:
#Create a boolean query using f-strings, and postgress casting
select_json3=f'select * from {existing_schema}.{new_table} where (info->>\'customerdiscount\')::boolean is true;'
print(select_json3)


In [ ]:
#IMPORTANT - (info->>'customerdiscount')::boolean
#Here we are returning a json text query from the table info,       info->>'customerdiscount'
#We are then wrapping the return in brackets (info->>'customerdiscount')
#And then we finaly cast the text to a boolean value using ::boolean
#
#(info->>'customerdiscount')::boolean
#
#We have covered casting using python, here we have used casting in SQL for the POSTGRES database.


In [ ]:
cur.execute(select_json3)
customers=cur.fetchall()

In [ ]:
for a_customer in customers:
    print(a_customer)